# Mars CTX Stereo Tool


# Imports

In [151]:
from shapely.geometry import shape

In [190]:
import geopandas as gp
from sidecar import Sidecar
import os
import json
from ipyleaflet import TileLayer, Map, DrawControl, SearchControl, Marker, AwesomeIcon, GeoJSON, GeoData, LayerGroup 
from ipyleaflet import WidgetControl, GeoJSON, basemap_to_tiles, LayersControl, projections, FullScreenControl
from ipywidgets import Text, HTML


# Defs

In [153]:
mars_eqc_crs = {'custom': True, 'name': 'Mars2000', 'proj4def': '+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +a=3396190 +b=3396190 +units=m +no_defs', }
mars_np_crs = {'custom': True, 'name': 'IAU2000:49919', 'proj4def': '+proj=stere +lat_0=90 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=3396190 +b=3376200 +units=m +no_defs', }
mars_sp_crs = {'custom': True, 'name': 'IAU2000:49920', 'proj4def': '+proj=stere +lat_0=-90 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=3396190 +b=3376200 +units=m +no_defs', }

In [154]:
ctx_fp_url = 'https://ode.rsl.wustl.edu/mars/datafile/derived_products/coverageshapefiles/mars/mro/ctx/edr/mars_mro_ctx_edr_c0a.zip'

In [155]:
ctx_df = gp.read_file(ctx_fp_url)

In [156]:
good_columns = ['ProductId','Ext2URL', 'EmAngle', 'InAngle', 'PhAngle']

In [243]:
mola_color_shade_url = 'https://astro.arcgis.com/arcgis/rest/services/OnMars/MColorDEM/MapServer/tile/{z}/{y}/{x}'
mola_color_bm = {
    'url': mola_color_shade_url,
    'attribution': 'USGS/ESRI/NASA',
    'crs':projections.EPSG4326,
    'max_native_zoom': 5,
}
muarry_ctx_global_mosaic = 'https://astro.arcgis.com/arcgis/rest/services/OnMars/CTX/MapServer/tile/{z}/{y}/{x}'
muarry_ctx_global_mosaic_bm = {
    'url': muarry_ctx_global_mosaic,
    'attribution': 'CalTech/USGS/MSSS/NASA',
    'crs':projections.EPSG4326,
    'max_native_zoom': 12,
}

In [244]:
def perform_stereo_query(df=ctx_df):
    # get query as shape
    bbox = shape(querys['features'][0]['geometry'])
    minx, miny, maxx, maxy = bbox.bounds
    # filter df down to everything intersecting the bbox
    df_fp1 = df.cx[minx:maxx, miny:maxy]
    # for now don't allow more than 100 ctx footprints, replace with something smarter later
    if len(df_fp1) > 100:
        df_fp1 = df_fp1.iloc[0:100]
    # perform more of query
    left = df_fp1.query('EmAngle < 5.0').copy()
    left.loc[:,'lg'] = left['geometry']
    right = df_fp1.query('EmAngle >= 5.0').copy()
    right.loc[:,'rg'] = right['geometry']
    # todo: drop any duplicates which shouldn't be possible but floats are bad
    df_fp2 = gp.overlay(left, right , how='intersection')
    df_fp2['diff_em'] = (df_fp2['EmAngle_2'] - df_fp2['EmAngle_1']).abs() 
    df_fp2['ovarea'] = df_fp2['geometry'].to_crs(mars_eqc_crs['proj4def']).area
    #
    # tt['ovarea_wl'] = gp.overlay(tt['geometry'], tt['lg'], how='intersection').area / tt['lg'].area * 100
    # tt['ovarea_wr'] = gp.overlay(tt['geometry'], tt['rg'], how='intersection').area / tt['rg'].area * 100
    # done!
    del df_fp2['lg']
    del df_fp2['rg']
    return df_fp2
    

In [245]:
good_query_columns = ['ProductId_1', 'ProductId_2', 'EmAngle_1', 'EmAngle_2', 'diff_em', 'ovarea', 'Ext2URL_1', 'Ext2URL_2',]

In [256]:
# make the map
m = Map(
    center=(0,0),
    zoom=1, 
    basemap=TileLayer(**mola_color_bm),
    crs=projections.EPSG4326,
)
m.add_layer(TileLayer(**muarry_ctx_global_mosaic_bm, name='CTX Global Mosaic'))
# hover widget 
html = HTML('''Hover over a Stereo Candidate''')
html.layout.margin = '0px 20px 20px 20px'
hovercontrol = WidgetControl(widget=html, position='topright')
m.add_control(hovercontrol)
# update html, needs to be used on the geodata.on_hover
def update_html(feature,  **kwargs):
    html.value = '''
        <h3><b>{}</b></h3>
        <a target="_blank" rel="noopener noreferrer" href="{}"><h4>LeftImg: {}</h4></a>
        <a target="_blank" rel="noopener noreferrer" href="{}"><h4>RightImg: {}</h4></a>
    '''.format(feature['properties']['diff_em'],
               feature['properties']['Ext2URL_1'],
               feature['properties']['ProductId_1'],
               feature['properties']['Ext2URL_2'],
               feature['properties']['ProductId_2'])
# full screen control
full_screen_control = FullScreenControl()
m.add_control(full_screen_control)



In [257]:
draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": .1
    }
}

querys = {
    'type': 'FeatureCollection',
    'features': [],
}
# current result store the df in 0, and the layer in 1
current_results = [None, None]
def on_selection(self, action, geo_json):
    global current_results, update_html
    if len(querys['features'])>=1:
        querys['features'] = list()
    querys['features'].append(geo_json)
    # perform query
    dfq = perform_stereo_query()
    current_results[0] = dfq 
    # update map
    if current_results[1] is not None:
        m.remove_layer(current_results[1])
    gd = GeoData(
        geo_dataframe=current_results[0][[*good_query_columns, 'geometry']],
        name='Current Results',
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    )
    gd.on_hover(update_html)
    current_results[1] = gd
    m.add_layer(current_results[1])
    
draw_control.on_draw(on_selection)
m.add_control(draw_control)
control = LayersControl(position='bottomleft')
m.add_control(control)


In [258]:
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [230]:
# finally show the map
sc = Sidecar(title='Mars')
with sc:
    display(m)

In [231]:
current_results[0][good_query_columns]

,ProductId_1,ProductId_2,EmAngle_1,EmAngle_2,diff_em,ovarea,Ext2URL_1,Ext2URL_2
0,P02_001876_1852_XI_05N010W,P02_002021_1848_XI_04N010W,1.86,23.23,21.37,6.680888e+08,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
1,F21_044006_1853_XN_05N010W,P02_002021_1848_XI_04N010W,0.10,23.23,23.13,1.056086e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
2,P02_001876_1852_XI_05N010W,B09_013097_1850_XN_05N010W,1.86,7.49,5.63,4.986608e+08,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
3,F21_044006_1853_XN_05N010W,B09_013097_1850_XN_05N010W,0.10,7.49,7.39,2.041229e+08,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
4,P02_001876_1852_XI_05N010W,D04_028736_1852_XI_05N010W,1.86,22.34,20.48,8.467770e+08,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
5,F21_044006_1853_XN_05N010W,D04_028736_1852_XI_05N010W,0.10,22.34,22.24,1.087817e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
6,P02_001876_1852_XI_05N010W,D04_028591_1864_XI_06N010W,1.86,8.45,6.59,6.924914e+08,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
7,F21_044006_1853_XN_05N010W,D04_028591_1864_XI_06N010W,0.10,8.45,8.35,1.039793e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
8,P02_001876_1852_XI_05N010W,D09_030582_1850_XN_05N010W,1.86,23.31,21.45,1.137208e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
9,F21_044006_1853_XN_05N010W,D09_030582_1850_XN_05N010W,0.10,23.31,23.21,4.497084e+08,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...


In [ ]:
query_string = 
"""
SELECT 
 a.ProductId as left_pi, 
 b.ProductId as right_pi,
 a.EmAngle as left_em,
 b.EmAngle as right_em,
 a.Ext2URL as left_url, 
 b.Ext2URL as right_url, 
 ABS(b.EmAngle - a.EmAngle)  as diff_em 
FROM 
mroctx as a, 
mroctx as b
WHERE
 a.EmAngle < 4
AND 
a.EmAngle < b.EmAngle 
AND
diff_em > 10 
AND
diff_em < 25
ORDER BY
ovarea desc,
diff_em desc,
left_em asc
LIMIT 1000;

"""